In [23]:
import pandas as pd
from sqlalchemy import create_engine

In [24]:
columns=['Merchant_ID','Order_ID','Response_ID','Auth_Code','ASV_Indicator','AVS_Result','CCV_Indicator'
         ,'CVV_Result','Payment_Method','Payment_Method_Name','Fraud_Result','External_Ref','Currency'
         ,'Amount','Expiration_Date','Auth_Date','Merchant_Number','Payment_Processor','Service_Account_Name'
         ,'Terminal_ID','Credit_Card_Company','Email_Address','Name','Street','House_Number','Additional_Address_Info'
         ,'Zip','City','Country_Code','State','BIN','Customer_ID','Merchant_Ref','Description','MCC','Vertical','Client_Name'
         ,'Interchange_Region','Status_ID','Status_Description','CVV_Description','AVS_Description']

In [25]:
df=pd.read_csv('d:\\djohnson\\Desktop\\conversion.txt',sep=';',skiprows=38,low_memory=True,encoding='ISO-8859-1',error_bad_lines=False,names=columns,index_col=False)

C:\Users\djohnson\new_python_version\Local\Continuum\anaconda2\envs\py3k\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (30,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
df['Description']=df['Description'].str.strip()
df['Client_Name']=df['Client_Name'].str.strip()
df['Status_Description']=df['Status_Description'].str.strip()

In [27]:
df[['Client_Name','Order_ID','Status_Description','Description']].head()

,Client_Name,Order_ID,Status_Description,Description
0,Linden Lab,102932637,Success,Approved or completed succesfully
1,Golden Frog,4000385972,Decline,Low funds
2,Golden Frog,4000388807,Success,Approved or completed succesfully
3,Golden Frog,4000388807,Success,Approved or completed succesfully
4,Golden Frog,4000388807,Success,Approved or completed succesfully


In [28]:
df_totals=df.groupby(['Merchant_ID','Client_Name','Status_Description','Description'],as_index=False)['Order_ID'].count()
df_grand_total=df.groupby(['Merchant_ID','Client_Name'],as_index=False)['Order_ID'].count()

In [29]:
df_totals.head(10)

,Merchant_ID,Client_Name,Status_Description,Description,Order_ID
0,149,"Teneroch SA de CV, Palace Holding",Success,Approved or completed succesfully,12
1,276,"Scalefast, Pepitastore",Success,Approved or completed succesfully,16
2,277,"Scalefast, Pepitastore",Success,,1
3,294,Gandi,Decline,CVV2 DECLINED,12
4,294,Gandi,Decline,Call issuer,3
5,294,Gandi,Decline,Do not honor,198
6,294,Gandi,Decline,Invalid card number,12
7,294,Gandi,Decline,Not permitted to cardholder,30
8,294,Gandi,Decline,Others,3
9,294,Gandi,Decline,Security rules not respected,42


In [30]:
df_totals_success=df_totals[['Merchant_ID','Client_Name','Order_ID']][(df_totals['Status_Description']=='Success')]
df_totals_success=df_totals_success.groupby(['Merchant_ID','Client_Name'],as_index=False)['Order_ID'].sum().copy()

In [31]:
df_totals_decline=df_totals[['Merchant_ID','Client_Name','Order_ID']][(df_totals['Status_Description']=='Decline')]
df_totals_decline=df_totals_decline.groupby(['Merchant_ID','Client_Name'],as_index=False)['Order_ID'].sum().copy()

In [32]:
df_totals_success.head()

,Merchant_ID,Client_Name,Order_ID
0,149,"Teneroch SA de CV, Palace Holding",12
1,276,"Scalefast, Pepitastore",16
2,277,"Scalefast, Pepitastore",1
3,294,Gandi,660
4,408,Swarovski Crystal Online AG,240


In [33]:
conversion_rate_perc=pd.merge(df_totals_success,df_grand_total,on=['Client_Name','Merchant_ID'],how='left').copy()

In [34]:
conversion_rate_perc=conversion_rate_perc.rename(columns={'Order_ID_x':'Success','Order_ID_y':'Total'})

In [35]:
conversion_rate_perc['Conversion_Success_Perc']=round((conversion_rate_perc['Success']/conversion_rate_perc['Total'])*100,0)

In [36]:
conversion_rate_perc.head()

,Merchant_ID,Client_Name,Success,Total,Conversion_Success_Perc
0,149,"Teneroch SA de CV, Palace Holding",12,12,100.0
1,276,"Scalefast, Pepitastore",16,16,100.0
2,277,"Scalefast, Pepitastore",1,1,100.0
3,294,Gandi,660,960,69.0
4,408,Swarovski Crystal Online AG,240,245,98.0


In [37]:
conversion_rate_perc=conversion_rate_perc[['Client_Name','Conversion_Success_Perc']]

In [38]:
conversion_rate_perc.describe()

,Conversion_Success_Perc
count,260.000000
mean,88.923077
std,17.461315
min,2.000000
25%,83.000000
50%,98.000000
75%,100.000000
max,100.000000


In [42]:
conversion_rate_perc=conversion_rate_perc['Conversion_Success_Perc'].value_counts()

KeyError: 'Conversion_Success_Perc'

In [40]:
conversion_rate_perc

100.0    123
95.0       8
93.0       7
97.0       6
89.0       5
83.0       5
79.0       5
86.0       5
92.0       5
96.0       5
99.0       4
70.0       4
80.0       4
88.0       4
98.0       4
87.0       4
94.0       4
76.0       3
81.0       3
67.0       3
50.0       3
90.0       3
71.0       3
69.0       2
75.0       2
84.0       2
91.0       2
73.0       2
64.0       2
68.0       2
77.0       2
78.0       2
85.0       2
82.0       1
53.0       1
8.0        1
56.0       1
63.0       1
15.0       1
54.0       1
2.0        1
37.0       1
59.0       1
62.0       1
19.0       1
51.0       1
66.0       1
74.0       1
41.0       1
32.0       1
60.0       1
31.0       1
40.0       1
Name: Conversion_Success_Perc, dtype: int64